In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta
from openpyxl import load_workbook

In [2]:
%run "./technical_analysis_self.ipynb"

In [3]:
%run "./yahoo_fetch.ipynb"

In [4]:
def get_returns(df):
    for i in [2, 3, 5, 8, 13]:
        df[f'd{i}_high'] = df['high'].shift(-i)
        df[f'd{i}_close'] = df['close'].shift(-i)
        df[f'd{i}_return_pct'] = (df[f'd{i}_close'] - df['next_day_open']) * 100 / df['next_day_open']
        df[f'd{i}_win_flag'] = df[f'd{i}_return_pct'] >= 2
    return df

def get_sliced_df(df, indicator_column):
    col_set = [indicator_column] + ['symbol', 'next_day_open', 'ADX', 'plus_DI', 'minus_DI', 
                                    'rsi', 'rsi_bin', 'bb_upper', 'bb_lower', 
                                    'bb_upper_distance_pct', 'bb_lower_distance_pct', 'bb_range'] 
    col_set += [i for i in df.columns if '_return_pct' in i or '_flag' in i]
    
    write_df = df[df[indicator_column].notna()][col_set]
    return write_df

def dump_to_excel(df, path, indicator_column):
    start_row = 0   
    write_df = get_sliced_df(df, indicator_column)
    
    try:
        wb = load_workbook(path)
        start_row = len(wb[indicator_column]['A'])
    except:
        pass
    
    try:
        with pd.ExcelWriter(path, mode='a', if_sheet_exists='overlay') as excel_writer:
            write_df.to_excel(excel_writer, sheet_name=indicator_column, startrow=start_row, header=start_row == 0) 
    except FileNotFoundError:
        write_df.to_excel(path, sheet_name=indicator_column, startrow=start_row) 

In [43]:
path_to_excel = './candlestick_stg_perf.xlsx'

In [5]:
excel_df = None

for script in ['infy', 'wipro', 'tcs', 'canbk', 'tatasteel', 'ashokley', 'oil', 'tejasnet', 'bhartiartl', 
               'hindpetro', 'railtel', 'zomato', 'hfcl', 'rain', 'paytm', 'biocon', 'bankbaroda', 'tatapower',
               'ambujacem', 'hcltech', 'sbin', 'tatamotors', 'coalindia', 'jswsteel', 'itc', 'hdfc'
              ]:
    df = get_historical_data(script, '2021-01-01', '2022-10-21').set_index('date')
    df["next_day_open"] = df["open"].shift(-1)
    df = load_ADX(load_ATR(df), 14)
    for fn in [preprocess, get_returns, identify_marubozus, identify_engulfing, identify_haramis, 
              identify_paper_umbrellas, identify_piercing_or_dark_clouds, identify_shooting_stars, load_rsi, load_bollinger_bands]:
        df = fn(df)
        
    excel_df = df if excel_df is None else pd.concat([excel_df, df])

if excel_df.index.name is None:
    excel_df = excel_df.set_index('date')

In [6]:
excel_df['rsi_bin'] = pd.cut(excel_df['rsi'], bins=[35, 40, 45, 50, 55, 60, 65])
excel_df['bb_range'] = excel_df['bb_upper'] - excel_df['bb_lower']
excel_df['bb_upper_distance_pct'] = (excel_df['bb_upper'] - excel_df['close']) / excel_df['bb_range']
excel_df['bb_lower_distance_pct'] = (excel_df['close'] - excel_df['bb_lower']) / excel_df['bb_range']

# start = datetime.now()
# for indicator_column in ['marubozu', 'engulfing', 'harami', 'partial_engulfing', 'paper_umbrella_type']:
#     dump_to_excel(excel_df, path_to_excel, indicator_column)

# print('Time Taken', datetime.now() - start)

In [14]:
engulfing_df = get_sliced_df(excel_df, 'engulfing')
engulfing_df[np.where(
    np.logical_or.reduce([
        (engulfing_df[f"d{i}_win_flag"] == True) for i in [2, 3, 5, 8, 13]
    ]),
    True,
    False
)].groupby("rsi_bin").agg({"rsi_bin": "count"})

AttributeError: 'SeriesGroupBy' object has no attribute 'percentage'

In [9]:
engulfing_df.shape

(733, 23)